In [1]:
import numpy as np
import pandas as pd
import pathlib as pl
import duckdb as ddb

In [2]:
filepath_closed_sales = pl.Path("T:", "Demand Planning", "Carry ATS review", "Exports", "Closed Sales")
closed_sales_file_name = {"ClosedSales": "Closed sales - size level.xlsx"}


filepath_closed_sales = pl.Path("T:", "Demand Planning", "Carry ATS review", "Exports", "Closed Sales", "Closed sales - size level.xlsx")

In [3]:
# closed_sales_data_1 = pd.read_excel(filepath_closed_sales, sheet_name = "Basics Sales Data")
# closed_sales_data_2 = pd.read_excel(filepath_closed_sales, sheet_name = "Basics Sales Data(1)")


In [4]:
ddb.sql("INSTALL excel; LOAD excel;")

In [5]:
# filepath_closed_sales = "T:\Demand Planning\Carry ATS review\Exports\Closed Sales\Closed sales - size level.xlsx"
filepath_closed_sales = "C:\\Users\luan.nguyen\Documents\monolith\planning_accuracy_report\__raw_data\Closed sales - size level.xlsx"

# closed_sales_data_1 = ddb.sql(f"""
#     SELECT * 
#     FROM read_xlsx("{filepath_closed_sales}", header = True, sheet = "Basics Sales Data")
#     WHERE "Date Start" >= '2024-10' AND "Date Start" <= '2024-12'
#     LIMIT 100
#     """)

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\luan.nguyen\AppData\Local\Temp\ipykernel_13040\518661839.py:2: SyntaxWarning: invalid escape sequence '\l'
  filepath_closed_sales = "C:\\Users\luan.nguyen\Documents\monolith\planning_accuracy_report\__raw_data\Closed sales - size level.xlsx"


In [6]:
# closed_sales_data_2 = ddb.sql(f"""
#     SELECT * 
#     FROM read_xlsx("{filepath_closed_sales}", header = false, sheet = "Basics Sales Data(1)") 
#     """)

In [7]:
## Load only the sheet into memory once
closed_sales_data = ddb.sql(f"""
    CREATE TEMPORARY TABLE closed_sales AS 
    SELECT * 
    FROM read_xlsx('{filepath_closed_sales}', header = TRUE, sheet = 'Basics Sales Data')
    UNION ALL
    SELECT * 
    FROM read_xlsx('{filepath_closed_sales}', header = TRUE, sheet = 'Basics Sales Data(1)')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
## Because we're query this data frequently, making it into parquet
ddb.sql(f"""
    COPY closed_sales TO '__raw_data/closed_sales.parquet' (FORMAT PARQUET)
""")

In [9]:
filepath_closed_sales_parquet = "C:\\Users\luan.nguyen\Documents\monolith\planning_accuracy_report\__raw_data\closed_sales.parquet"

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\luan.nguyen\AppData\Local\Temp\ipykernel_13040\2011546860.py:1: SyntaxWarning: invalid escape sequence '\l'
  filepath_closed_sales_parquet = "C:\\Users\luan.nguyen\Documents\monolith\planning_accuracy_report\__raw_data\closed_sales.parquet"


In [ ]:
## Get total count of rows to make sure data is intact
## As of 2025-08-24
count = ddb.sql(f"""
    SELECT COUNT(*)
    FROM '{filepath_closed_sales_parquet}'
""").fetchone()[0]

print(count)

1297875


In [16]:
closed_sales_data_1 = ddb.sql(f"""
    SELECT *
    FROM '{filepath_closed_sales_parquet}'
    WHERE "Date Start" >= '2024-10' AND "Date Start" <= '2024-12'
""")

In [17]:
count_2024 = ddb.sql(f"""
    SELECT COUNT(*)
    FROM 'closed_sales_data_1'
""").fetchone()[0]

print(count_2024)

464570


In [18]:
closed_sales_data_2 = ddb.sql(f"""
    SELECT *
    FROM '{filepath_closed_sales_parquet}'
    WHERE "Date Start" >= '2025-01' AND "Date Start" <= '2025-10'
""")

In [19]:
count_2025 = ddb.sql(f"""
    SELECT COUNT(*)
    FROM 'closed_sales_data_2'
""").fetchone()[0]

print(count_2025)

833305


In [21]:
with pd.ExcelWriter("__raw_data/closed_sales.xlsx") as writer:
    closed_sales_data_1.df().to_excel(writer, sheet_name="202410-202412", index = False)
    closed_sales_data_2.df().to_excel(writer, sheet_name="202501-202509", index = False)